# Funwave-tvd Visualization

The first cell code will read the funwave-tvd input file, and get some basic parameters for visualization.
Please execute the below code firstly, then you could generate plots separately. 

In [ ]:
import numpy as np
import re
import os
home = os.environ["HOME"]
run_dir = os.path.join(home, "work", "run_dir")

# Set the output directory 
opdir = os.path.join(run_dir, "output")

# Set some basic parameters 
para_value_pairs = {
    "Mglob"      : "",    # global dimension in x direction
    "Nglob"      : "",    # global dimension in y direction
    "TOTAL_TIME" : "",    # simulation time in seconds
    "PLOT_INTV"  : "",    # output interval in seconds
    "DX"         : "",    # grid size(m) in x direction
    "DY"         : ""     # grid size(m) in y direction
}

# Parse input file, and get the value of parameters
with open(os.path.join(run_dir,"input.txt"), "r") as fd:
    for line in fd.readlines():
        g = re.search('(\w+)\s*=\s*(\S+)', line)
        if g:
            name = g.group(1)
            val = g.group(2)
            if name in para_value_pairs:
                para_value_pairs[name] = val
                print(name,"->",val)
    
Mglob = int(para_value_pairs['Mglob'])
Nglob = int(para_value_pairs['Nglob'])
TOTAL_TIME = float(para_value_pairs['TOTAL_TIME'])
PLOT_INTV = float(para_value_pairs['PLOT_INTV'])
DX = float(para_value_pairs['DX'])
DY = float(para_value_pairs['DY'])

# Calculate the amount of frames of outputs
total_frame = int(TOTAL_TIME/PLOT_INTV + 1)



In [ ]:
# Look at the output directory
!ls {opdir}

## Time Series 1D
Here you could generate time series 1D plot with output variables, including elevation, u, v, Hmax, Hmix...., on specific location. 
Just change the location_x, location_y, output_var and unit.

In [ ]:
import matplotlib.pyplot as plt

# Revise the location_x in the range of 1 to Nglob
# Revise the location_y in the range of 1 to Mglob
location_x = 2
location_y = 300

# Revise the output variable for Y axis of plots
output_var = "eta" 

# Revise the unit of output variable
unit ='(m)'


# Draw 1D plot 
def OneD(location_x, location_y):
    plt.clf()
    plt.cla()
    plt.close()
    
    fig, ax = plt.subplots()
    
    X = []
    Y = []
    
    try: 
        for frame in range(total_frame):
            file_name = os.path.join(opdir,output_var+'_%05d' % (frame+1))
            if not os.path.exists(file_name):
                continue
            y = np.loadtxt(file_name)
            Y.append(y[location_x][location_y])
            time = '%.2f' % ((frame)*PLOT_INTV)
            X.append(time)

        ax.plot(X,Y)
        ax.set_title(f"At location ({location_x})({location_y})")
        ax.set_ylabel(output_var+" "+unit)
        ax.set_xlabel("Time (s)")
        plt.show()
        
    finally:
        plt.close()

# Show plot
OneD(location_x, location_y)

## Individual Snapshot 
Here you could generate plot with any individual snapshot, including elevation, u, v and depth. 
Just change the frame and plot_title variables.

In [ ]:
import matplotlib.pyplot as plt1
from matplotlib.colors import LinearSegmentedColormap

# Change the frame that you want to show
frame = "eta_00010"

# Revise the title of the plot
plot_title = "Surface elevation (m)"

# Custom color map, have to add two or two more colors 
# Here are some example, just uncomment one of below colors

colors=["#5bd6a0","#5b93db", "#78969f"]
#colors=["#c29284", "#c2d0aa", "#ceddf1"]
#colors=["#7addf1", "#7a89f1", "#ce8ca3"]

# Generate color map
cmap = LinearSegmentedColormap.from_list("", colors)

# Draw 2D plot
def twoDsnapPlot(frame):    
    
    X_value = [ [ ((i+1)*DX) for i in range(Mglob) ] for j in range(Nglob) ]
    Y_value = [ [ ((j+1)*DY) for i in range(Mglob) ] for j in range(Nglob) ]
    ax, fig = plt.subplots()
    eta = np.loadtxt(os.path.join(opdir,frame))
    etaplot = plt1.contourf(X_value, Y_value, eta, 100, cmap = cmap)
    plt1.title(plot_title)
    plt1.colorbar()

    plt1.xlabel("X (m)")
    plt1.ylabel("Y (m)")
    plt1.tight_layout()
    plt1.show()

# Show plot
twoDsnapPlot(frame)

In [ ]:
# Plot the depth file
depth_data = np.genfromtxt(os.path.join(opdir,"dep.out"))
plt.pcolor(depth_data)

##  Animation 2D
Here you could generate 2D animation with part of or all of results, including elevation, u and v...  Just change the start_time, end_time and output_var. 

In [ ]:
def load_times(vname):
    times = []
    for fn in os.listdir(opdir):
        g = re.match(vname+r"_(\d+)", fn)
        if g:
            times += [int(g.group(1))]
    return sorted(times)

In [ ]:
import matplotlib.pyplot as plt2
from matplotlib import animation
from IPython.display import HTML
from matplotlib.colors import LinearSegmentedColormap

# Revise the start time and end time, not exceed TOTAL_TIME
# Note: start_time have to be less than end_time
start_time = "19"
end_time = "20.0"

# Revise the output variable for Y axis of plots
output_var = "eta"

times = load_times(output_var)

# Custom color map, have to add two or two more colors 
# Here are some example, just uncomment one of below colors

colors=["#5bd6a0","#5b93db", "#78969f"]
#colors=["#c29284", "#c2d0aa", "#ceddf1"]
#colors=["#7addf1", "#7a89f1", "#ce8ca3"]

# Generate color map
cmap = LinearSegmentedColormap.from_list("", colors)

# Draw 2D snapshot animation
def twoDsnapAnim(start, end):
    #fig = plt2.figure()
    fig, ax = plt2.subplots()
    
    X_value = [ [ str((i+1)*DX) for i in range(Mglob) ] for j in range(Nglob) ]
    Y_value = [ [ str((j+1)*DY) for i in range(Mglob) ] for j in range(Nglob) ]
    
    fig_size = []
    fig_size.append(8)
    fig_size.append(5)
    plt2.rcParams["figure.figsize"] = fig_size
    
    def animate(i):
        ax.clear()
        try:
            file_name = os.path.join(opdir,output_var+'_%05d' % (i))
            Eta = np.loadtxt(file_name)
            img = ax.contourf(X_value, Y_value, Eta, 100, cmap = cmap)
            plt.colorbar(img)
            plt.close()
            time = '%.2f' % ((i+1)*(PLOT_INTV))
            ax.set_title('Time = '+ time +' sec')
            ax.set_xlabel("X (m)")
            ax.set_ylabel("Y (m)")
            return ax
        finally:
            plt2.close()

    # make animate 
    try: 
        anim = animation.FuncAnimation(fig, animate, times, 
                                   interval=200,blit=False)
        display(HTML(anim.to_html5_video()))
    finally:
        plt2.close()

# Show animation
twoDsnapAnim(start_time, end_time)

## Cross-shore Profile animation 2D
Here you could generate 2D cross-shore profile animation with elevation results. Just change start_time, end_time and depth_loc. 

In [ ]:
import matplotlib.pyplot as plt3
from matplotlib import animation
from IPython.display import HTML

# Revise depth_loc in the range of 1 to Nglob
depth_loc = 1

# Revise the start time and end time, not exceed TOTAL_TIME
# Note: start_time have to be less than end_time
start_time = "14"
end_time = "15"

output_var = "eta"

# Change the water color and base color
waterColor = '#87CEFA'
baseColor  = '#EEE8AA'
    
# Draw 2D cross-shore profile animation
def depProfileWithEta(depth, start, end):
    # open depth file 
    depthfile = open(os.path.join(opdir,'dep.out'),'r')
    depthdata = depthfile.readlines()[int(depth)-1]
    p = depthdata.split()

    # load depth data to dep_value
    dep_value = []
    for i in range(int(Mglob)):
        dep_value.append(-abs(float(p[i])))

    # create x_value
    x_value = []
    for i in range(int(Mglob)):
        x_value.append((i+1)*DX)
    
    #fig = plt.figure()
    fig, ax = plt3.subplots()

    # create etas that contains all of needing eta data
    etas = []
    length = len(times)
    for i in range(length):
        file_name = os.path.join(opdir,'%s_%05d' % (output_var, times[i]))
        eta_value = np.genfromtxt(file_name)[depth,:]
        etas.append(eta_value)

    # define animate function
    def animate(i):
        ax.clear()
        ax.plot(x_value, etas[i], color = waterColor)
        ax.plot(x_value, dep_value, color = baseColor)
        ax.fill_between(x_value,-1, etas[i], facecolor = waterColor)
        ax.fill_between(x_value,-1, dep_value, facecolor = baseColor)
        time = '%.2f' % (times[i]*PLOT_INTV)
        ax.set_title('Time = '+ time +' sec')
        #plt.ylim(-0.45,0.1)
        #plt.xlabel("X (m)")
        #plt.ylabel("Elevation (m)")
        plt3.close()
        ax.set_xlabel("X (m)")
        ax.set_ylabel("Elevation (m)")
        ax.set_ylim(-0.45,0.1)
        
        return ax

    # make animate 
    anim = animation.FuncAnimation(fig, animate, range(length),
                                   interval=200, blit=False)
    display(HTML(anim.to_html5_video()))
    
# Show animation
depProfileWithEta(depth_loc, start_time, end_time)